In [1]:
import os
import sys

import numpy as np
import tensorflow as tf
import dataIO as d

from tqdm import *
from utils import *
import pickle

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import pickle
# %matplotlib inline


import urllib.request
response = urllib.request.urlopen("https://raw.githubusercontent.com/yaroslavvb/memory_util/master/memory_util.py")
open("memory_util.py", "wb").write(response.read())

import memory_util
memory_util.vlog(1)

All dependencies not loaded, some functionality may not work


In [2]:
batch_size = 100
z_size = 200
cube_len = 32
leak_value = 0.2
weights = {}
l = 0.95
lr = 0.001

obj = 'vase'
obj_ratio = 1.0
num_epoch = 101
save_interval = 5
beta       = 0.5

experiment_name = 'vase-compress_3_test'

voxnet_save_path = '/home/vamsi/Downloads/tf-3dgan-master/src/models/cls_vox/biasfree_19999.cptk'
gan_save_directory = '/home/vamsi/Downloads/tf-3dgan-master/src/models/vase/biasfree_5600.cptk'

train_sample_directory = './train_sample/' + experiment_name + '/'
model_directory = './models/' + experiment_name + '/'

img_base_directory = './img/'
img_directory = img_base_directory + experiment_name + '/'

pickle_base_directory = './pickle/'
pickle_directory = pickle_base_directory + experiment_name + '/'

if not os.path.exists(pickle_directory):
    os.makedirs(pickle_directory)
    
if not os.path.exists(train_sample_directory):
    os.makedirs(train_sample_directory)
    
if not os.path.exists(model_directory):
    os.makedirs(model_directory)
    
if not os.path.exists(img_directory):
    os.makedirs(img_directory)

In [3]:
def decoder(z, batch_size=batch_size, phase_train=True, reuse=False):

    strides    = [2,2,2]

    with tf.variable_scope("gen", reuse=reuse):
#         z = tf.reshape(z, (batch_size, 1, 1, 1, z_size))
        g_1 = tf.layers.dense(z, 256*2*2*2, kernel_initializer=tf.random_normal_initializer(stddev=0.02), name='g1',trainable=phase_train)
        g_1 = tf.reshape(g_1, (-1, 2,2,2,256))
#         g_1 = tf.nn.conv3d_transpose(z, weights['wg1'], (batch_size,4,4,4,512), strides=[1,1,1,1,1], padding="VALID")
        g_1 = tf.layers.batch_normalization(g_1, training=phase_train, name='g_bn1')
        g_1 = tf.nn.relu(g_1)

        g_2 = tf.layers.conv3d_transpose(g_1, 256, [4, 4, 4], strides=strides, padding="same", use_bias=False, name='g2',trainable=phase_train)
        g_2 = tf.layers.batch_normalization(g_2, training=phase_train, name='g_bn2')
        g_2 = tf.nn.relu(g_2)

        g_3 = tf.layers.conv3d_transpose(g_2, 128, [4, 4, 4], strides=strides, padding="same", use_bias=False, name='g3',trainable=phase_train)
        g_3 = tf.layers.batch_normalization(g_3, training=phase_train, name='g_bn3')
        g_3 = tf.nn.relu(g_3)

        g_4 = tf.layers.conv3d_transpose(g_3, 64, [4, 4, 4], strides=strides, padding="same", use_bias=False, name='g4',trainable=phase_train)
        g_4 = tf.layers.batch_normalization(g_4, training=phase_train, name='g_bn4')
        g_4 = tf.nn.relu(g_4)
        
        g_5 = tf.layers.conv3d_transpose(g_4, 1, [4, 4, 4], strides=strides, padding="same", use_bias=False, name='g5',trainable=phase_train)
        g_5 = tf.nn.sigmoid(g_5)
#         g_5 = tf.nn.tanh(g_5)

    print (g_1, 'g1')
    print (g_2, 'g2')
    print (g_3, 'g3')
    print (g_4, 'g4')
    print (g_5, 'g5')
    
    return g_5

def encoder(inputs, encoding_size, phase_train=True, reuse=False):

    strides    = [2,2,2]
    with tf.variable_scope("enc", reuse=reuse):
             
        d_1 = tf.layers.conv3d(inputs, 32, [4, 4, 4], strides=strides, padding="same", use_bias=False, name='e1',trainable=phase_train)
        d_1 = tf.layers.batch_normalization(d_1, training=phase_train, name='bn_e1')                               
        d_1 = lrelu(d_1, leak_value)

        d_2 = tf.layers.conv3d(d_1, 64, [4, 4, 4], strides=strides, padding="same", use_bias=False, name='e2',trainable=phase_train) 
        d_2 = tf.layers.batch_normalization(d_2, training=phase_train, name='bn_e2')
        d_2 = lrelu(d_2, leak_value)
        
        d_3 = tf.layers.conv3d(d_2, 128, [4, 4, 4], strides=strides, padding="same", use_bias=False, name='e3',trainable=phase_train)
        d_3 = tf.layers.batch_normalization(d_3, training=phase_train, name='bn_e3')
        d_3 = lrelu(d_3, leak_value) 

        d_4 = tf.layers.conv3d(d_3, 256, [4, 4, 4], strides=strides, padding="same", use_bias=False, name='e4',trainable=phase_train)    
        d_4 = tf.layers.batch_normalization(d_4, training=phase_train, name='bn_e4')
        d_4 = lrelu(d_4)

        d_5 = tf.contrib.layers.flatten(d_4)
        d_5 = tf.layers.dense(d_5, encoding_size, kernel_initializer=tf.random_normal_initializer(stddev=0.02), name='f1',trainable=phase_train)
        d_5_no_sigmoid = d_5
        d_5 = tf.nn.sigmoid(d_5)
        
        
    print (d_1, 'd1')
    print (d_2, 'd2')
    print (d_3, 'd3')
    print (d_4, 'd4')
    print (d_5, 'd5')

    return d_5, d_5_no_sigmoid
        


In [4]:
def lrelu(x, leak=0.2):
    return tf.maximum(x, leak*x)

def voxnet(input_models, phase_train=False, reuse=False):
    strides    = [2,2,2]
    with tf.variable_scope("vox", reuse=reuse):
        vox_1 = tf.layers.conv3d(input_models, 32, [5, 5, 5], strides=strides, padding="valid", trainable = phase_train, use_bias=False, name='vox1')
        vox_1 = tf.layers.batch_normalization(vox_1, training=phase_train, name='vox_bn1', trainable = phase_train)                               
        vox_1 = lrelu(vox_1, leak_value)
        
        vox_2 = tf.layers.conv3d(vox_1, 32, [3, 3, 3], strides=[1,1,1], padding="valid", use_bias=False, name='vox2', trainable = phase_train)
        vox_2 = tf.layers.batch_normalization(vox_2, training=phase_train, name='vox_bn2', trainable = phase_train)                               
        vox_2 = lrelu(vox_2, leak_value)
        vox_2 = tf.layers.max_pooling3d(vox_2, [2,2,2], strides=[1,1,1], padding='valid', name='max_pool1')
        
        vox_flat = tf.contrib.layers.flatten(vox_2)
        vox_3 = tf.layers.dense(vox_flat, 5, kernel_initializer=tf.random_normal_initializer(stddev=0.02), name='vox3', trainable = phase_train)
        vox_3_no_softmax = vox_3
        vox_3 = tf.nn.softmax(vox_3_no_softmax)

    #return 2nd layer of voxnet
    return vox_2

In [5]:
def plot_voxel(voxel, voxel2=None, saveas=None):
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    ax.voxels(voxel.squeeze()>.5, facecolors='red', edgecolors='k')
    ax.set_axis_off()

    if voxel2 is not None:
        ax = fig.add_subplot(1, 2, 2, projection='3d')
        ax.voxels(voxel2.squeeze()>.5, facecolors='red', edgecolors='k')
        ax.set_axis_off()
        
    if saveas is not None:
        plt.savefig(saveas)

#     plt.show()

In [6]:

    
x_vector = tf.placeholder(shape=[batch_size,cube_len,cube_len,cube_len,1],dtype=tf.float32) 

#encoding from 32*32*32 down to 200 ::: 163x
enc,_ = encoder(x_vector, z_size, phase_train=True, reuse=False)
dec = decoder(enc, phase_train=True, reuse=False) 

input_vox = voxnet(x_vector, phase_train=False, reuse=False)
decoded_vox = voxnet(dec, phase_train=False, reuse=True)

num_voxnet_intermediate_units = np.product(input_vox.get_shape().as_list())

loss_pixel = tf.nn.l2_loss(x_vector - dec) / (batch_size*cube_len*cube_len*cube_len)
loss_perceptual = tf.nn.l2_loss(input_vox - decoded_vox) / num_voxnet_intermediate_units
# weighted sum of the two losses
loss = l*loss_pixel + (1-l)*loss_perceptual

/home/vamsi/miniconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensor("enc/Maximum:0", shape=(100, 16, 16, 16, 32), dtype=float32) d1
Tensor("enc/Maximum_1:0", shape=(100, 8, 8, 8, 64), dtype=float32) d2
Tensor("enc/Maximum_2:0", shape=(100, 4, 4, 4, 128), dtype=float32) d3
Tensor("enc/Maximum_3:0", shape=(100, 2, 2, 2, 256), dtype=float32) d4
Tensor("enc/Sigmoid:0", shape=(100, 200), dtype=float32) d5
Tensor("gen/Relu:0", shape=(100, 2, 2, 2, 256), dtype=float32) g1
Tensor("gen/Relu_1:0", shape=(100, 4, 4, 4, 256), dtype=float32) g2
Tensor("gen/Relu_2:0", shape=(100, 8, 8, 8, 128), dtype=float32) g3
Tensor("gen/Relu_3:0", shape=(100, 16, 16, 16, 64), dtype=float32) g4
Tensor("gen/Sigmoid:0", shape=(100, 32, 32, 32, 1), dtype=float32) g5


In [7]:
encoder_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='enc')
#build a list of variable to load
generator_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='gen')
#generator_vars.extend(list(weights.values()))

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_op = tf.train.AdamOptimizer(learning_rate=lr, beta1 = beta ).minimize(loss)

In [8]:

code_directory = '/home/vamsi/Downloads/tf-3dgan-master/src/models/vase-compress_3/compression_gen_100.ckpt'

with tf.Session() as sess:
    print('Helllllllllllllllllllllllllllllllllllllllllllllllllllllllllllo')

    sess.run(tf.global_variables_initializer())

    print('Heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy')

    #all_saver = tf.train.Saver()
    saver = tf.train.Saver()

    gen_loader = tf.train.Saver(generator_vars)
    gen_loader.restore(sess, code_directory)

    vox_loader = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='vox'))
    vox_loader.restore(sess, voxnet_save_path)
    
    ec_loader = tf.train.Saver(encoder_vars)
    ec_loader.restore(sess, code_directory)

    # load from checkpoint
    # all_saver.restore(sess, tf.train.latest_checkpoint(model_directory))
    volumes = d.getAll(obj=obj, train=True, is_local=False, obj_ratio=obj_ratio, cube_len=32)
    print ('Using ' + obj + ' Data')
    volumes = volumes[...,np.newaxis].astype(np.float)

    num_samples = len(volumes)

    num_batches = num_samples // batch_size

    #sess.run(tf.global_variables_initializer())   
    loss_stat = np.array([])
    for epoch in range(100, 200):
        for i in range(num_batches):
            idx = np.random.randint(len(volumes), size=batch_size)
            input_voxels = volumes[idx]
            pix_loss, perc_loss, tot_loss, _ = sess.run([loss_pixel, loss_perceptual, loss, train_op], feed_dict={x_vector: input_voxels})

            loss_stat = np.concatenate([loss_stat,[tot_loss]])

            print("epoch {}, batch {}: pix loss {:10.5f}, perc loss {:10.5f}, total loss {:10.5f}".format(epoch, i, pix_loss, perc_loss, tot_loss))
            if i==num_batches - 1 and epoch % save_interval == 0:
                # generate and visualize generated images
                recon_voxel = sess.run(dec, feed_dict={x_vector: input_voxels})
                save_sample = np.concatenate((input_voxels, recon_voxel), axis=0)

                save_sample.dump("{}{}".format(train_sample_directory, epoch))

                saver.save(sess, save_path = '{}compression_gen_{}.ckpt'.format(model_directory, epoch))

                for n in range(3):
                    plot_voxel(save_sample[n], save_sample[n + batch_size], saveas='{}{}_{}.png'.format(img_directory, epoch, n))

                loss_stat.dump("{}{}.pickle".format(pickle_directory, epoch))
                
                volumes_test = d.getAll(obj=obj, train=False, is_local=False, obj_ratio=obj_ratio, cube_len=32)
    
                volumes_test = volumes_test[...,np.newaxis].astype(np.float)

                num_samples = len(volumes_test)

                print(num_samples)

                num_batches = num_samples // batch_size

                sess.run(tf.global_variables_initializer())  

                loss_stat = np.array([])

                for epoch in range(20):
                    for i in range(num_batches):
                        input_voxels = volumes[i*batch_size: (i+1)*batch_size]
                        # generate and visualize generated images
                        pix_loss, recon_voxel = sess.run([loss_pixel, dec], feed_dict={x_vector: input_voxels})
                        #print("epoch {}, batch {}: pix loss {:10.5f}, perc loss {:10.5f}, total loss {:10.5f}".format(epoch, i, pix_loss, perc_loss, tot_loss))
                    save_sample = np.concatenate((input_voxels, recon_voxel), axis=0)
                    recon_voxel.dump(train_sample_directory+'/biasfree_'+str(epoch))
                    for n in range(3):
                        plot_voxel(save_sample[n], save_sample[n + batch_size], saveas='{}{}_{}_test.png'.format(img_directory, epoch, n))

                
                
                

Helllllllllllllllllllllllllllllllllllllllllllllllllllllllllllo
Heyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
INFO:tensorflow:Restoring parameters from /home/vamsi/Downloads/tf-3dgan-master/src/models/vase-compress_3/compression_gen_100.ckpt
INFO:tensorflow:Restoring parameters from /home/vamsi/Downloads/tf-3dgan-master/src/models/cls_vox/biasfree_19999.cptk
INFO:tensorflow:Restoring parameters from /home/vamsi/Downloads/tf-3dgan-master/src/models/vase-compress_3/compression_gen_100.ckpt
Using vase Data
epoch 100, batch 0: pix loss    0.00263, perc loss    0.00232, total loss    0.00262
epoch 100, batch 1: pix loss    0.00335, perc loss    0.00274, total loss    0.00331
epoch 100, batch 2: pix loss    0.00236, perc loss    0.00173, total loss    0.00233
epoch 100, batch 3: pix loss    0.00354, perc loss    0.00311, total loss    0.00352
epoch 100, batch 4: pix loss    0.00274, perc loss    0.00228, total loss    0.00272
epoch 100, batch 5: pix loss    0.00303, perc loss    0.00267, total loss    0.0

epoch 101, batch 33: pix loss    0.00175, perc loss    0.00135, total loss    0.00173
epoch 101, batch 34: pix loss    0.00245, perc loss    0.00213, total loss    0.00243
epoch 101, batch 35: pix loss    0.00246, perc loss    0.00194, total loss    0.00244
epoch 101, batch 36: pix loss    0.00236, perc loss    0.00225, total loss    0.00235
epoch 101, batch 37: pix loss    0.00230, perc loss    0.00213, total loss    0.00230
epoch 101, batch 38: pix loss    0.00290, perc loss    0.00247, total loss    0.00288
epoch 101, batch 39: pix loss    0.00210, perc loss    0.00199, total loss    0.00210
epoch 101, batch 40: pix loss    0.00242, perc loss    0.00194, total loss    0.00240
epoch 101, batch 41: pix loss    0.00230, perc loss    0.00215, total loss    0.00229
epoch 101, batch 42: pix loss    0.00181, perc loss    0.00141, total loss    0.00179
epoch 101, batch 43: pix loss    0.00278, perc loss    0.00233, total loss    0.00276
epoch 101, batch 44: pix loss    0.00185, perc loss   

epoch 103, batch 15: pix loss    0.00220, perc loss    0.00166, total loss    0.00218
epoch 103, batch 16: pix loss    0.00144, perc loss    0.00115, total loss    0.00143
epoch 103, batch 17: pix loss    0.00243, perc loss    0.00209, total loss    0.00241
epoch 103, batch 18: pix loss    0.00217, perc loss    0.00192, total loss    0.00216
epoch 103, batch 19: pix loss    0.00177, perc loss    0.00140, total loss    0.00175
epoch 103, batch 20: pix loss    0.00209, perc loss    0.00159, total loss    0.00207
epoch 103, batch 21: pix loss    0.00181, perc loss    0.00155, total loss    0.00180
epoch 103, batch 22: pix loss    0.00250, perc loss    0.00207, total loss    0.00247
epoch 103, batch 23: pix loss    0.00211, perc loss    0.00175, total loss    0.00209
epoch 103, batch 24: pix loss    0.00182, perc loss    0.00149, total loss    0.00180
epoch 103, batch 25: pix loss    0.00228, perc loss    0.00167, total loss    0.00225
epoch 103, batch 26: pix loss    0.00197, perc loss   

epoch 104, batch 54: pix loss    0.00207, perc loss    0.00163, total loss    0.00205
epoch 104, batch 55: pix loss    0.00264, perc loss    0.00204, total loss    0.00261
epoch 104, batch 56: pix loss    0.00204, perc loss    0.00177, total loss    0.00202
epoch 105, batch 0: pix loss    0.00147, perc loss    0.00120, total loss    0.00145
epoch 105, batch 1: pix loss    0.00181, perc loss    0.00162, total loss    0.00180
epoch 105, batch 2: pix loss    0.00201, perc loss    0.00165, total loss    0.00199
epoch 105, batch 3: pix loss    0.00155, perc loss    0.00132, total loss    0.00154
epoch 105, batch 4: pix loss    0.00158, perc loss    0.00125, total loss    0.00156
epoch 105, batch 5: pix loss    0.00212, perc loss    0.00156, total loss    0.00210
epoch 105, batch 6: pix loss    0.00226, perc loss    0.00180, total loss    0.00224
epoch 105, batch 7: pix loss    0.00125, perc loss    0.00095, total loss    0.00124
epoch 105, batch 8: pix loss    0.00201, perc loss    0.00174,

epoch 106, batch 36: pix loss    0.00262, perc loss    0.00231, total loss    0.00260
epoch 106, batch 37: pix loss    0.00245, perc loss    0.00190, total loss    0.00242
epoch 106, batch 38: pix loss    0.00226, perc loss    0.00169, total loss    0.00224
epoch 106, batch 39: pix loss    0.00179, perc loss    0.00138, total loss    0.00177
epoch 106, batch 40: pix loss    0.00171, perc loss    0.00134, total loss    0.00169
epoch 106, batch 41: pix loss    0.00179, perc loss    0.00146, total loss    0.00178
epoch 106, batch 42: pix loss    0.00231, perc loss    0.00192, total loss    0.00229
epoch 106, batch 43: pix loss    0.00195, perc loss    0.00157, total loss    0.00193
epoch 106, batch 44: pix loss    0.00154, perc loss    0.00107, total loss    0.00152
epoch 106, batch 45: pix loss    0.00210, perc loss    0.00168, total loss    0.00208
epoch 106, batch 46: pix loss    0.00160, perc loss    0.00140, total loss    0.00159
epoch 106, batch 47: pix loss    0.00195, perc loss   

...

epoch 191, batch 36: pix loss    0.00073, perc loss    0.00057, total loss    0.00073
epoch 191, batch 37: pix loss    0.00106, perc loss    0.00092, total loss    0.00106
epoch 191, batch 38: pix loss    0.00103, perc loss    0.00082, total loss    0.00102
epoch 191, batch 39: pix loss    0.00070, perc loss    0.00053, total loss    0.00069
epoch 191, batch 40: pix loss    0.00093, perc loss    0.00077, total loss    0.00093
epoch 191, batch 41: pix loss    0.00106, perc loss    0.00079, total loss    0.00105
epoch 191, batch 42: pix loss    0.00073, perc loss    0.00058, total loss    0.00072
epoch 191, batch 43: pix loss    0.00081, perc loss    0.00061, total loss    0.00080
epoch 191, batch 44: pix loss    0.00103, perc loss    0.00080, total loss    0.00102
epoch 191, batch 45: pix loss    0.00081, perc loss    0.00063, total loss    0.00080
epoch 191, batch 46: pix loss    0.00094, perc loss    0.00078, total loss    0.00093
epoch 191, batch 47: pix loss    0.00078, perc loss   

epoch 193, batch 18: pix loss    0.00084, perc loss    0.00065, total loss    0.00083
epoch 193, batch 19: pix loss    0.00069, perc loss    0.00050, total loss    0.00068
epoch 193, batch 20: pix loss    0.00093, perc loss    0.00073, total loss    0.00092
epoch 193, batch 21: pix loss    0.00107, perc loss    0.00091, total loss    0.00106
epoch 193, batch 22: pix loss    0.00080, perc loss    0.00065, total loss    0.00079
epoch 193, batch 23: pix loss    0.00072, perc loss    0.00055, total loss    0.00071
epoch 193, batch 24: pix loss    0.00104, perc loss    0.00085, total loss    0.00103
epoch 193, batch 25: pix loss    0.00118, perc loss    0.00097, total loss    0.00117
epoch 193, batch 26: pix loss    0.00102, perc loss    0.00082, total loss    0.00101
epoch 193, batch 27: pix loss    0.00087, perc loss    0.00074, total loss    0.00087
epoch 193, batch 28: pix loss    0.00079, perc loss    0.00061, total loss    0.00078
epoch 193, batch 29: pix loss    0.00092, perc loss   

epoch 195, batch 0: pix loss    0.00109, perc loss    0.00087, total loss    0.00107
epoch 195, batch 1: pix loss    0.00118, perc loss    0.00099, total loss    0.00117
epoch 195, batch 2: pix loss    0.00055, perc loss    0.00043, total loss    0.00055
epoch 195, batch 3: pix loss    0.00128, perc loss    0.00105, total loss    0.00126
epoch 195, batch 4: pix loss    0.00094, perc loss    0.00081, total loss    0.00093
epoch 195, batch 5: pix loss    0.00099, perc loss    0.00074, total loss    0.00098
epoch 195, batch 6: pix loss    0.00090, perc loss    0.00073, total loss    0.00089
epoch 195, batch 7: pix loss    0.00098, perc loss    0.00084, total loss    0.00098
epoch 195, batch 8: pix loss    0.00083, perc loss    0.00061, total loss    0.00082
epoch 195, batch 9: pix loss    0.00082, perc loss    0.00067, total loss    0.00082
epoch 195, batch 10: pix loss    0.00085, perc loss    0.00062, total loss    0.00084
epoch 195, batch 11: pix loss    0.00108, perc loss    0.00080, 

epoch 196, batch 39: pix loss    0.00084, perc loss    0.00058, total loss    0.00082
epoch 196, batch 40: pix loss    0.00067, perc loss    0.00049, total loss    0.00066
epoch 196, batch 41: pix loss    0.00093, perc loss    0.00072, total loss    0.00092
epoch 196, batch 42: pix loss    0.00076, perc loss    0.00052, total loss    0.00075
epoch 196, batch 43: pix loss    0.00076, perc loss    0.00052, total loss    0.00074
epoch 196, batch 44: pix loss    0.00115, perc loss    0.00084, total loss    0.00113
epoch 196, batch 45: pix loss    0.00109, perc loss    0.00081, total loss    0.00107
epoch 196, batch 46: pix loss    0.00115, perc loss    0.00090, total loss    0.00114
epoch 196, batch 47: pix loss    0.00115, perc loss    0.00097, total loss    0.00114
epoch 196, batch 48: pix loss    0.00082, perc loss    0.00066, total loss    0.00081
epoch 196, batch 49: pix loss    0.00085, perc loss    0.00064, total loss    0.00084
epoch 196, batch 50: pix loss    0.00079, perc loss   

epoch 198, batch 21: pix loss    0.00078, perc loss    0.00063, total loss    0.00077
epoch 198, batch 22: pix loss    0.00092, perc loss    0.00064, total loss    0.00090
epoch 198, batch 23: pix loss    0.00084, perc loss    0.00065, total loss    0.00083
epoch 198, batch 24: pix loss    0.00079, perc loss    0.00064, total loss    0.00079
epoch 198, batch 25: pix loss    0.00097, perc loss    0.00081, total loss    0.00097
epoch 198, batch 26: pix loss    0.00084, perc loss    0.00062, total loss    0.00083
epoch 198, batch 27: pix loss    0.00085, perc loss    0.00064, total loss    0.00084
epoch 198, batch 28: pix loss    0.00070, perc loss    0.00055, total loss    0.00069
epoch 198, batch 29: pix loss    0.00073, perc loss    0.00053, total loss    0.00072
epoch 198, batch 30: pix loss    0.00077, perc loss    0.00062, total loss    0.00076
epoch 198, batch 31: pix loss    0.00077, perc loss    0.00058, total loss    0.00076
epoch 198, batch 32: pix loss    0.00097, perc loss   

In [6]:
x_vector = tf.placeholder(shape=[batch_size,cube_len,cube_len,cube_len,1],dtype=tf.float32) 

#encoding from 32*32*32 down to 200 ::: 163x
enc,_ = encoder(x_vector, z_size, phase_train=False, reuse=False)
dec = decoder(enc, phase_train=False, reuse=False) 

input_vox = voxnet(x_vector, phase_train=False, reuse=False)
decoded_vox = voxnet(dec, phase_train=False, reuse=True)

num_voxnet_intermediate_units = np.product(input_vox.get_shape().as_list())

loss_pixel = tf.nn.l2_loss(x_vector - dec) / (batch_size*cube_len*cube_len*cube_len)
loss_perceptual = tf.nn.l2_loss(input_vox - decoded_vox) / num_voxnet_intermediate_units
# weighted sum of the two losses
loss = l*loss_pixel + (1-l)*loss_perceptual

/home/vamsi/miniconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensor("enc/Maximum:0", shape=(100, 16, 16, 16, 32), dtype=float32) d1
Tensor("enc/Maximum_1:0", shape=(100, 8, 8, 8, 64), dtype=float32) d2
Tensor("enc/Maximum_2:0", shape=(100, 4, 4, 4, 128), dtype=float32) d3
Tensor("enc/Maximum_3:0", shape=(100, 2, 2, 2, 256), dtype=float32) d4
Tensor("enc/Sigmoid:0", shape=(100, 200), dtype=float32) d5
Tensor("gen/Relu:0", shape=(100, 2, 2, 2, 256), dtype=float32) g1
Tensor("gen/Relu_1:0", shape=(100, 4, 4, 4, 256), dtype=float32) g2
Tensor("gen/Relu_2:0", shape=(100, 8, 8, 8, 128), dtype=float32) g3
Tensor("gen/Relu_3:0", shape=(100, 16, 16, 16, 64), dtype=float32) g4
Tensor("gen/Sigmoid:0", shape=(100, 32, 32, 32, 1), dtype=float32) g5


In [7]:
encoder_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='enc')
#build a list of variable to load
generator_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='gen')
#generator_vars.extend(list(weights.values()))

In [ ]:
code_directory = '/home/vamsi/Downloads/tf-3dgan-master/src/models/vase-compress_3/compression_gen_90.ckpt'

with tf.Session() as sess:
    
    #sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()

    gen_loader = tf.train.Saver(generator_vars)
    gen_loader.restore(sess, code_directory)

    ec_loader = tf.train.Saver(encoder_vars)
    ec_loader.restore(sess, code_directory)

    vox_loader = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='vox'))
    vox_loader.restore(sess, voxnet_save_path)

    # change the path 
    volumes = d.getAll(obj=obj, train=False, is_local=False, obj_ratio=obj_ratio, cube_len=32)
    
    volumes = volumes[...,np.newaxis].astype(np.float)

    num_samples = len(volumes)
    
    print(num_samples)

    num_batches = num_samples // batch_size


    loss_stat = np.array([])

    for epoch in range(20):
        for i in range(num_batches):
            input_voxels = volumes[i*batch_size: (i+1)*batch_size]
            # generate and visualize generated images
            pix_loss, perc_loss, tot_loss, recon_voxel = sess.run([loss_pixel, loss_perceptual, loss, dec], feed_dict={x_vector: input_voxels})
            print("epoch {}, batch {}: pix loss {:10.5f}, perc loss {:10.5f}, total loss {:10.5f}".format(epoch, i, pix_loss, perc_loss, tot_loss))
            for n in range(3):
                plot_voxel(save_sample[n], save_sample[n + batch_size], saveas='{}{}_{}_test.png'.format(img_directory, epoch*i, n))

        #save_sample = np.concatenate((input_voxels, recon_voxel), axis=0)
        #recon_voxel.dump(train_sample_directory+'/biasfree_'+str(epoch))
        


INFO:tensorflow:Restoring parameters from /home/vamsi/Downloads/tf-3dgan-master/src/models/vase-compress_3/compression_gen_90.ckpt
INFO:tensorflow:Restoring parameters from /home/vamsi/Downloads/tf-3dgan-master/src/models/vase-compress_3/compression_gen_90.ckpt
INFO:tensorflow:Restoring parameters from /home/vamsi/Downloads/tf-3dgan-master/src/models/cls_vox/biasfree_19999.cptk
1200
